## 6.1 텍스트 파일 이용

In [3]:
import pandas as pd
import numpy as np

In [1]:
!cat ch06/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
df = pd.read_csv('ch06/ex1.csv')

In [5]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_table('ch06/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


컬럼이 없는 데이터의 경우

In [7]:
!cat ch06/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [8]:
pd.read_csv('ch06/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
pd.read_csv('ch06/ex2.csv', names=['a','b','c','d','message']) # names : column 명

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
names = ['a','b','c','d','message']
pd.read_csv('ch06/ex2.csv', names=names, index_col='message') # message 컬럼을 index로 활용

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


계층적 색인

In [14]:
!cat ch06/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [15]:
parsed = pd.read_csv('ch06/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

공백이나 다른 구분자로 구분한 경우 read_table

In [16]:
list(open('ch06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [18]:
result = pd.read_table('ch06/ex3.txt', sep='\s+') # 공백문자
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


파일에 데이터외의 다른 내용이 있을경우 제외할 로우 선택

In [19]:
!cat ch06/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [20]:
pd.read_csv('ch06/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


누락된 값 처리

In [21]:
!cat ch06/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [22]:
result = pd.read_csv('ch06/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [32]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [30]:
# na_values 옵션
result = pd.read_csv('ch06/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [31]:
sentinels = {'message' : ['foo', 'NA'], 'something' : ['two']}
pd.read_csv('ch06/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


참고(read_csv parameter) : http://pandas.pydata.org/pandas-docs/version/0.18/generated/pandas.read_csv.html

### 6.1.1 텍스트 파일 조금씩 읽어오기

In [33]:
result = pd.read_csv('ch06/ex6.csv', nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [39]:
# 파일을 여러조각으로 나누기
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
# tot = tot.order(ascending=False)
tot = tot.sort_values(ascending=False)

In [41]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 6.1.2 데이터를 텍스트 형식으로 기록

In [42]:
data = pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [44]:
data.to_csv('ch06/out.csv')

In [45]:
!cat ch06/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [47]:
import sys

In [48]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [49]:
# row와 행의 이름이 없이
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [50]:
# series to_csv
dates = pd.date_range('1/1/2017', periods=7)
ts = pd.Series(np.arange(7), index=dates)

In [51]:
ts.to_csv('ch06/tseries.csv')

In [52]:
!cat ch06/tseries.csv

2017-01-01,0
2017-01-02,1
2017-01-03,2
2017-01-04,3
2017-01-05,4
2017-01-06,5
2017-01-07,6


In [53]:
# series from csv
pd.Series.from_csv('ch06/tseries.csv', parse_dates=True)

2017-01-01    0
2017-01-02    1
2017-01-03    2
2017-01-04    3
2017-01-05    4
2017-01-06    5
2017-01-07    6
dtype: int64

### 6.1.3 수동으로 구분 형식 처리하기

In [54]:
!cat ch06/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [56]:
import csv
f = open('ch06/ex7.csv')
reader = csv.reader(f)
for line in reader: # 한줄씩
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [62]:
for h, v in zip(header, zip(*values)):
    print(h," : ", v)

a  :  ('1', '1')
b  :  ('2', '2')
c  :  ('3', '3')


In [57]:
lines = list(csv.reader(open('ch06/ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h : v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [66]:
# 다양한 구분자 처리 클래스
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
    
f = open('ch06/ex7.csv')
reader = csv.reader(f, dialect=my_dialect)
reader

### 6.1.4 JSON 데이터

In [68]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null, "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
     {"name": "Katie", "age": 33, "pet": "cisco"}]

}
"""

In [69]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'cisco'}]}

In [72]:
# python object to json structure
asjson = json.dumps(result)
type(asjson)

str

In [73]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,25
1,Katie,33


### 6.1.5 XML과 HTML: 웹 내용 긁어오기

In [75]:
from lxml.html import parse
from urllib.request import urlopen 
# from urllib2 import urlopen (python 2.x)

parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=APPL+Options'))
doc = parsed.getroot()
doc

<Element html at 0x10b21ccc8>

In [76]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x10c8b11d8>,
 <Element a at 0x10c8b1228>,
 <Element a at 0x10c8b1278>,
 <Element a at 0x10c8b12c8>,
 <Element a at 0x10c8b1318>]

In [77]:
lnk = links[28]
print(lnk)
print(lnk.get('href'))
print(lnk.text_content())

<Element a at 0x10c8b15e8>
/calendar
Markets


In [78]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['/',
 '/watchlists',
 '/portfolios?bypass=true',
 '/screener',
 '/calendar',
 '/industries',
 '/personal-finance',
 '/tech',
 '/topic/yahoo-finance-podcast',
 '/live/facebook']

In [81]:
tables = doc.findall('.//table')
tables

[]

lxml.objectify를 이용한 xml 파싱 

In [82]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot() 
root
# 예제 따라할 경우 에러(Performance_MNR.xml 이 없음)

FileNotFoundError: [Errno 2] No such file or directory: 'Performance_MNR.xml'

## 6.2 이진 데이터 형식

In [90]:
frame = pd.read_csv('ch06/ex1.csv')
type(frame)

pandas.core.frame.DataFrame

In [92]:
dir(frame)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__

In [93]:
frame.to_pickle('ch06/frame_pickle')

In [94]:
pd.read_pickle('ch06/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 6.2.1 HDF5 형식 사용

In [96]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

ImportError: HDFStore requires PyTables, "No module named 'tables'" problem importing

### 6.2.2 마이크로소프트 엑셀 파일 읽기

In [101]:
xls_file = pd.read_excel('ch06/data.xls')

FileNotFoundError: [Errno 2] No such file or directory: 'ch06/data.xls'

## 6.3 HTML, 웹 API와 함께 사용(책의 내용과 다름)

In [183]:
data = open('twitter.json').read()

In [184]:
data = json.loads(data)

In [187]:
CONSUMER_KEY=data['CONSUMER_KEY']
CONSUMER_SECRET=data['CONSUMER_SECRET']
ACCESS_TOKEN_KEY=data['ACCESS_TOKEN_KEY']
ACCESS_TOKEN_SECRET=data['ACCESS_TOKEN_SECRET']

In [170]:
import requests
from requests_oauthlib import OAuth1
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)
requests.get(url, auth=auth)

<Response [200]>

In [178]:
r = requests.get('https://api.twitter.com/1.1/search/tweets.json?q=python%20pandas', auth=auth)
data = json.loads(r.text)
data.keys()

dict_keys(['statuses', 'search_metadata'])

In [179]:
data['statuses']

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Sat Apr 21 14:30:01 +0000 2018',
  'entities': {'hashtags': [{'indices': [0, 6], 'text': 'udemy'},
    {'indices': [7, 17], 'text': 'udemyfree'},
    {'indices': [18, 30], 'text': 'udemycoupon'},
    {'indices': [31, 45], 'text': 'onlineclasses'},
    {'indices': [46, 60], 'text': 'onlinecourses'},
    {'indices': [61, 75], 'text': 'UdemyDiscount'},
    {'indices': [87, 100], 'text': 'DataAnalysis'},
    {'indices': [106, 113], 'text': 'Pandas'}],
   'symbols': [],
   'urls': [{'display_url': 'twitter.com/i/web/status/9…',
     'expanded_url': 'https://twitter.com/i/web/status/987699982483378177',
     'indices': [117, 140],
     'url': 'https://t.co/jfbJ5cY9Xx'}],
   'user_mentions': []},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 987699982483378177,
  'id_str': '987699982483378177',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_repl

In [180]:
tweet_fields = ['created_at' ,'from_user', 'id', 'text']
tweets = pd.DataFrame(data['statuses'], columns=tweet_fields)

In [181]:
tweets

,created_at,from_user,id,text
0,Sat Apr 21 14:30:01 +0000 2018,NaN,987699982483378177,#udemy #udemyfree #udemycoupon #onlineclasses ...
1,Sat Apr 21 14:18:34 +0000 2018,NaN,987697102145900545,RT @PythonHub: Reading large csv taking too mu...
2,Sat Apr 21 14:11:07 +0000 2018,NaN,987695229053620224,Reading large csv taking too much time. [Panda...
3,Sat Apr 21 13:42:18 +0000 2018,NaN,987687975428481027,Python For Data Science For Dummies (for Dummi...
4,Sat Apr 21 13:42:17 +0000 2018,NaN,987687973863948288,Mastering Pandas For Finance https://t.co/aZc...
5,Sat Apr 21 12:22:48 +0000 2018,NaN,987667969248514048,Cython: A Guide For Python Programmers https:...
6,Sat Apr 21 11:42:43 +0000 2018,NaN,987657880525565953,Data Analytics With Python: Data Analytics In ...
7,Sat Apr 21 11:14:53 +0000 2018,NaN,987650878873194496,RT @python_tip: Pass functions to .loc[] and ....
8,Sat Apr 21 11:02:03 +0000 2018,NaN,987647647153467396,RT @The_Academy_BOT: Python for Data Analysis:...
9,Sat Apr 21 11:02:02 +0000 2018,NaN,987647642686586881,Python for Data Analysis: Data Wrangling With ...


## 6.4 데이터베이스와 함께 사용

In [114]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER); 
"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [117]:
data = [('Atlanta', 'Georgia', 1.25, 6), ('Tallahassee', 'Florida', 2.6, 3), ('Sancramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test values(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [118]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sancramento', 'California', 1.7, 5)]

In [120]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [122]:
print(zip(*cursor.description))

In [127]:
pd.DataFrame(rows, columns=list(zip(*cursor.description))[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Atlanta,Georgia,1.25,6
2,Tallahassee,Florida,2.60,3
3,Sancramento,California,1.70,5


In [131]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Atlanta,Georgia,1.25,6
2,Tallahassee,Florida,2.60,3
3,Sancramento,California,1.70,5
